In [ ]:
import numpy as np
import pandas as pd
import io
import re
from konlpy.tag import Hannanum, Kkma, Komoran, Mecab, Okt
from gensim.models import Word2Vec


In [ ]:
data = pd.read_csv('/Users/minhyuk/study/example/korean_drinks_food.csv')
data.head()


In [ ]:
shop_list = data[['전통주명', '잘 어울리는 음식']]
shop_list.head()


In [ ]:
df = pd.DataFrame(shop_list, columns=['전통주명', '잘 어울리는 음식'])
df.head()


In [ ]:
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()


In [ ]:
from tqdm import tqdm


noun_list = []
for content in tqdm(df['잘 어울리는 음식']):
    nouns = okt.nouns(content)
    noun_list.append(nouns)

df['nouns'] = noun_list
df.head()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

text = [" ".join(noun) for noun in df['nouns']]

tfidf_vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

vector = np.array(vector)
model = DBSCAN(eps=0.4, min_samples=5, metric="cosine")
result = model.fit_predict(vector)

df['result'] = result
df.head()


In [ ]:
for cluster_num in set(result):
    if (cluster_num == -1 or cluster_num == 0):
        continue
    else:
        print("cluster num : {}".format(cluster_num))
        temp_df = df[df['result'] == cluster_num]
        for menu in temp_df['잘 어울리는 음식']:
            print(menu)
        print()
